#Face Swapping in Videos

## Initial Configuration and Setup

### Package Installation

In [ ]:
!pip install scikit-video

###Importing Libraries

In [ ]:
import dlib
import cv2
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import os
import shutil
import math
from scipy.spatial import Delaunay
import skvideo.io

###Mounting Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!cp -r '/content/gdrive/Shareddrives/CIS 581- Face Swapping In Videos/project/model' /content
!cp -r '/content/gdrive/Shareddrives/CIS 581- Face Swapping In Videos/project/videos/src' /content
!cp -r '/content/gdrive/Shareddrives/CIS 581- Face Swapping In Videos/project/faceBlendCommon.py' /content

## Initialization of Global Variables

In [ ]:
MODEL_PATH = '/content/gdrive/Shareddrives/CIS 581- Face Swapping In Videos/project/model/'
PREDICTOR_PATH = MODEL_PATH + "shape_predictor_68_face_landmarks.dat"
RESIZE_HEIGHT = 480
NUM_FRAMES_FOR_FPS = 100
SKIP_FRAMES = 1
detector = dlib.get_frontal_face_detector()
landmarkDetector = dlib.shape_predictor(PREDICTOR_PATH)

## Inter Eye Distance Calculation for Stabilization

In [ ]:
def interEyeDistance(predict):
  leftEyeLeftCorner = (predict[36].x, predict[36].y)
  rightEyeRightCorner = (predict[45].x, predict[45].y)
  distance = cv2.norm(np.array(rightEyeRightCorner) - np.array(leftEyeLeftCorner))
  distance = int(distance)
  return distance

## Face and Facial Landmark Detection with Stabilization

In [ ]:
def stabilise(im):
  points=[]
  pointsPrev=[]
  pointsDetectedCur=[]
  pointsDetectedPrev=[]
  all_stabilized_frames=[]

  eyeDistanceNotCalculated = True
  eyeDistance = 0
  isFirstFrame = True
  fps = 10
  showStabilized = False
  count =0

  imDlib = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
  imGray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
  clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8)) # improve image contrast
  imGray = clahe.apply(imGray)
  imGrayPrev = imGray

  height = im.shape[0]

  IMAGE_RESIZE = float(height)/RESIZE_HEIGHT
  imSmall = cv2.resize(im, None, fx=1.0/IMAGE_RESIZE, fy=1.0/IMAGE_RESIZE,interpolation = cv2.INTER_LINEAR)
  imSmallDlib = cv2.cvtColor(imSmall, cv2.COLOR_BGR2RGB)

  faces = detector(imSmallDlib,0)

  if len(faces)==0:
    pass

  else:
    for i in range(0,len(faces)):
        newRect = dlib.rectangle(int(faces[i].left() * IMAGE_RESIZE),
          int(faces[i].top() * IMAGE_RESIZE),
          int(faces[i].right() * IMAGE_RESIZE),
          int(faces[i].bottom() * IMAGE_RESIZE))
        
        landmarks = landmarkDetector(imDlib, newRect).parts()

        if (isFirstFrame==True):
          pointsPrev=[]
          pointsDetectedPrev = []
          [pointsPrev.append((p.x, p.y)) for p in landmarks]
          [pointsDetectedPrev.append((p.x, p.y)) for p in landmarks]
        else:
          pointsPrev=[]
          pointsDetectedPrev = []
          pointsPrev = points
          pointsDetectedPrev = pointsDetectedCur

        points = []
        pointsDetectedCur = []
        [points.append((p.x, p.y)) for p in landmarks]
        [pointsDetectedCur.append((p.x, p.y)) for p in landmarks]
        pointsArr = np.array(points,np.float32)
        pointsPrevArr = np.array(pointsPrev,np.float32)

        if eyeDistanceNotCalculated:
          eyeDistance = interEyeDistance(landmarks)
          eyeDistanceNotCalculated = False

        if eyeDistance > 100:
            dotRadius = 3
        else:
          dotRadius = 2

        sigma = eyeDistance * eyeDistance / 400
        s = 2*int(eyeDistance/4)+1
        lk_params = dict(winSize  = (s, s), maxLevel = 5, criteria = (cv2.TERM_CRITERIA_COUNT | cv2.TERM_CRITERIA_EPS, 20, 0.03))
        pointsArr,status, err = cv2.calcOpticalFlowPyrLK(imGrayPrev,imGray,pointsPrevArr,pointsArr,**lk_params)
        pointsArrFloat = np.array(pointsArr,np.float32)
        points = pointsArrFloat.tolist()

        for k in range(0,len(landmarks)):
          d = cv2.norm(np.array(pointsDetectedPrev[k]) - np.array(pointsDetectedCur[k]))
          alpha = math.exp(-d*d/sigma)
          points[k] = (1 - alpha) * np.array(pointsDetectedCur[k]) + alpha * np.array(points[k])
    return points

##Convex Hull Creation

In [ ]:
def convex_hull(points, points1):
  vec = np.empty([68,2], dtype=int)
  for b in range(68):
    vec[b][0] = points[b][0]
    vec[b][1] = points[b][1]

  indices = cv2.convexHull(np.array(vec), returnPoints = False)
  hull1 = vec[indices[:,0]]
  vec1 = np.empty([68,2], dtype=int)
  for b in range(68):
    vec1[b][0] = points1[b][0]
    vec1[b][1] = points1[b][1]
  hull2 = vec1[indices[:,0]]
  return hull1, hull2

## Warping and Affine Tranformation

In [ ]:
def warpTriangle(img1, img2, tri1, tri2):
    rect1 = cv2.boundingRect(np.float32([tri1]))
    rect2 = cv2.boundingRect(np.float32([tri2]))

    tri1Rect = np.zeros((3,2))
    tri2Rect = np.zeros((3,2))

    for i in range(0, 3):
        tri1Rect[i] = ((tri1[i][0] - rect1[0]), (tri1[i][1] - rect1[1]))
        tri2Rect[i] = (((tri2[i][0] - rect2[0]), (tri2[i][1] - rect2[1])))

    mask = np.zeros((rect2[3], rect2[2], 3), dtype=np.float32)
    cv2.fillConvexPoly(mask, np.int32(tri2Rect), (1.0, 1.0, 1.0), 16, 0);

    img1Rect = img1[rect1[1]:rect1[1] + rect1[3], rect1[0]:rect1[0] + rect1[2]]

    size = (rect2[2], rect2[3])

    affineWarped = cv2.getAffineTransform(np.float32(tri1Rect), np.float32(tri2Rect))
    img2Rect = cv2.warpAffine(img1Rect, affineWarped, (size[0], size[1]), None, flags=cv2.INTER_LINEAR,borderMode=cv2.BORDER_REFLECT_101)

    img2Rect = img2Rect * mask

    img2[rect2[1]:rect2[1] + rect2[3], rect2[0]:rect2[0] + rect2[2]] = img2[rect2[1]:rect2[1] + rect2[3], rect2[0]:rect2[0] + rect2[2]] * ((1.0, 1.0, 1.0) - mask)
    img2[rect2[1]:rect2[1] + rect2[3], rect2[0]:rect2[0] + rect2[2]] = img2[rect2[1]:rect2[1] + rect2[3], rect2[0]:rect2[0] + rect2[2]] + img2Rect

## Delaunay Triangulation

In [ ]:
def triangulation(hull1, hull2, img1, img1Warped):

  img2 = np.copy(img1Warped)

  tri = Delaunay(hull2).simplices

  numTriangles = len(tri)
  for i in range(0, numTriangles):
    triangle = tri[i]
    pts1 = np.zeros((3, 2), dtype=np.float32)
    pts2 = np.zeros((3, 2), dtype=np.float32)

    pts1[0] = hull1[triangle[0]]
    pts1[1] = hull1[triangle[1]]
    pts1[2] = hull1[triangle[2]]

    pts2[0] = hull2[triangle[0]]
    pts2[1] = hull2[triangle[1]]
    pts2[2] = hull2[triangle[2]]

    warpTriangle(img1, img1Warped, pts1, pts2)

  mask = np.zeros(img1.shape, dtype=img1.dtype)
  cv2.fillConvexPoly(mask, np.int32(hull2), (255, 255, 255))

  r = cv2.boundingRect(np.float32([hull2]))
  faceCenter = (r[0]+int(r[2]/2), r[1]+int(r[3]/2))

  finalImage = cv2.seamlessClone(np.uint8(img1Warped), np.uint8(img2), mask, faceCenter, cv2.NORMAL_CLONE)

  return cv2.cvtColor(np.uint8(finalImage), cv2.COLOR_BGR2RGB)

## Video Reading and Initiation of the Face Swapping Procedure

In [ ]:
video_path = '/content/gdrive/Shareddrives/CIS 581- Face Swapping In Videos/project/videos/src/FrankUnderwood.mp4'
video_name = video_path.split("/")[-1].split(".")[0]
cap= cv2.VideoCapture(video_path)
pic, frame = cap.read()
pic=True
frames1 = []
while(pic):
    pic, frame = cap.read()
    if pic==True:
        frames1.append(frame)
    else:
        break
cap.release()

video_path = '/content/gdrive/Shareddrives/CIS 581- Face Swapping In Videos/project/videos/target/FrankUnderwood- target.mp4'
video_name = video_path.split("/")[-1].split(".")[0]
cap= cv2.VideoCapture(video_path)
pic, frame = cap.read()
pic=True
frames2=[]
while(pic):
    pic, frame = cap.read()
    if pic==True:
        frames2.append(frame)
    else:
        break
cap.release()

plt.imshow(frames1[5][:,:,::-1])

[h1, w1, _] = frames1[0].shape
swappedVid1 = np.zeros((len(frames1) - 1, h1, w1, 3), dtype=np.uint8)

for i in range(len(frames1) - 1):
    if i < len(frames2) - 1: 
        img1 = frames2[i]
    else:
        img1 = frames2[len(frames2) - 2]
    points=stabilise(img1)
    points1=stabilise(frames1[i])
    if(points is None or points1 is None):
      continue
    hull1,hull2=convex_hull(points,points1)
    img2=np.copy(frames1[i])
    swappedVid1[i,:,:,:] = triangulation(hull1,hull2, img1, img2 )
    

[h2, w2, _] = frames2[0].shape
swappedVid2 = np.zeros((len(frames2) - 1, h2, w2, 3), dtype=np.uint8)

for j in range(len(frames2) - 1):
    if j < len(frames1) - 1: 
        img1 = frames1[j]
    else:
        img1 = frames1[len(frames1) - 2]
    points=stabilise(img1)
    points1=stabilise(frames2[j])
    if(points is None or points1 is None):
      continue
    hull1,hull2=convex_hull(points,points1)
    img2=np.copy(frames2[j])

    swappedVid2[j,:,:,:] = triangulation(hull1,hull2, img1, img2)
   

skvideo.io.vwrite("face2_on_video1.mp4", swappedVid1)
skvideo.io.vwrite("face1_on_video2.mp4", swappedVid2)